In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from delta.tables import *

In [0]:
%sql
--DROP DATABASE IF EXISTS ADVENTUREWORKS_SILVER CASCADE

In [0]:
%sql
CREATE DATABASE IF NOT EXISTS ADVENTUREWORKS_SILVER
LOCATION '/mnt/02-silver/ADVENTUREWORKS/'

In [0]:
%python

# Lista todos os diretórios dentro do container 01-bronze
dirs = [f.path for f in dbutils.fs.ls("/mnt/01-bronze/ADVENTUREWORKS") if f.isDir()]
dataframes = {}

# Para cada diretório encontrado, obtém o arquivo mais recente e armazena o caminho na variável correspondente
for d in dirs:
    # Obtém o nome do diretório
    dir_name = d.split("/")[4]

    # Obtém o caminho do arquivo mais recente dentro do diretório
    files = dbutils.fs.ls(d)
    latest_file = sorted(files, key=lambda f: f.modificationTime)[-1]
    file_path = latest_file.path
    
    # Armazena o caminho da origem na variável correspondente
    globals()[f"bronze_path_{dir_name}"] = file_path

    #Lê dados do arquivo mais recente
    df = spark.read.parquet(globals()[f"bronze_path_{dir_name}"])

    #Cria um dataframe para cada arquivo
    df_name = "DF_" + dir_name
    dataframes[df_name] = df

    #Atualiza as tabelas na camada Silver do Delta Lake
    dataframes[df_name].write\
    .mode("overwrite")\
    .option("overwriteSchema","true")\
    .saveAsTable("ADVENTUREWORKS_SILVER.TB_"+dir_name)
